In [ ]:
import numpy as np
import tensorflow as tf
import sys
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from logger import logging
from exception import CustomException 

# -------------------------------
# Hyperparameters
# -------------------------------
VOCAB_SIZE = 5000
SEQ_LEN = 1000
EMBED_DIM = 128
LSTM_DIM = 64
NUM_LAYERS = 2
LR = 1e-3

# -------------------------------
# Sample Dataset (Modify as needed)
# -------------------------------
X_train_sequences = np.random.randint(1, VOCAB_SIZE, (13731, SEQ_LEN))
X_test_sequences = np.random.randint(1, VOCAB_SIZE, (2000, SEQ_LEN))
X_train_other_features = np.random.rand(13731, 10)  # 10 additional features
X_test_other_features = np.random.rand(2000, 10)

y_train_sequences = np.random.randint(0, 5, (13731, 1))  # Labels between 0-4
y_test_sequences = np.random.randint(0, 5, (2000, 1))

# -------------------------------
# Fix: Ensure Labels are Correctly Shaped
# -------------------------------
y_train_sequences = y_train_sequences.reshape(-1)  # (13731,)
y_test_sequences = y_test_sequences.reshape(-1)  # (2000,)

# -------------------------------
# Define the Model
# -------------------------------
try:
    def build_model(vocab_size, seq_len, embed_dim, lstm_dim, num_layers, lr):
        # Text Input (Sequences)
        sequence_input = Input(shape=(seq_len,), name="sequence_input")
        embedding = Embedding(vocab_size, embed_dim, mask_zero=True)(sequence_input)
        x = LSTM(lstm_dim, return_sequences=False)(embedding)
        x = Dropout(0.3)(x)
        
        # Other Features Input
        other_input = Input(shape=(10,), name="other_input")
        other_dense = Dense(32, activation="relu")(other_input)

        # Merge both inputs
        concatenated = Concatenate()([x, other_dense])
        output = Dense(5, activation="softmax")(concatenated)  # 5 classes

        # Create Model
        model = Model(inputs=[sequence_input, other_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        return model

    # -------------------------------
    # Build and Train the Model
    # -------------------------------
    model = build_model(VOCAB_SIZE, SEQ_LEN, EMBED_DIM, LSTM_DIM, NUM_LAYERS, LR)
    model.summary()

    print("X_train_sequences shape:", X_train_sequences.shape)
    print("X_train_other_features shape:", X_train_other_features.shape)
    print("y_train_sequences shape:", y_train_sequences.shape)

    # Train Model
    history = model.fit(
        [X_train_sequences, X_train_other_features], 
        y_train_sequences,
        validation_data=([X_test_sequences, X_test_other_features], y_test_sequences),
        epochs=10,
        batch_size=64
    )

    # Evaluate Model
    test_loss, test_accuracy = model.evaluate([X_test_sequences, X_test_other_features], y_test_sequences)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
except Exception as e:
    raise CustomException(e, sys)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 1000)]       0           []                               
                                                                                                  
 embedding_11 (Embedding)       (None, 1000, 128)    640000      ['sequence_input[0][0]']         
                                                                                                  
 lstm_21 (LSTM)                 (None, 64)           49408       ['embedding_11[0][0]']           
                                                                                                  
 other_input (InputLayer)       [(None, 10)]         0           []                               
                                                                                            

In [14]:
# Save the entire model
model.save("lstm_model.h5")
print("Model saved as lstm_model.h5")


Model saved as lstm_model.h5
